In [3]:
import tensorflow as tf
import os
import pandas as pd
import boto3
from sklearn.model_selection import train_test_split
!pip install SimpleITK
import SimpleITK as sitk
!pip install plotly
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [ ]:
can = pd.read_csv("candidates_V2.csv")
can = can.drop(columns=["coordX", "coordY", "coordZ"])
can = can.drop_duplicates(subset="seriesuid")

In [ ]:
s3client = boto3.client('s3')
bucket = 'fastvision.ai'

s3objects= s3client.list_objects_v2(Bucket=bucket, Prefix="segmented_data/LUNA16_segmented_2mm/" )
files = []
for object in s3objects['Contents']:
    split = object['Key'].split("/")
    if ".mhd" in split[-1]:
        files.append(split[-1])

In [ ]:
files = [file[:-4] for file in files]
metadata = pd.DataFrame(files, columns=["seriesuid"])
joined = pd.merge(metadata, can, how='left')

In [ ]:
train, train = train_test_split(joined, test_size=0.3, random_state=42, stratify=joined[['class']])

In [ ]:
val, test = train_test_split(val_test, test_size=0.5, random_state=42, stratify=val_test[['class']])

In [ ]:
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
train.to_csv("train.csv")
val.to_csv("val.csv")
test.to_csv("test.csv")

In [4]:
metadata_test = pd.read_csv("metadata_test.csv")
metadata_test = metadata_test[metadata_test.seriesuid != "1.3.6.1.4.1.14519.5.2.1.6279.6001.100620385482151095585000946543"]

In [5]:
data_dir = "."
def process(data, label):
    file_path = os.path.join(data_dir, data.numpy().decode("utf-8") + ".mhd")
    ct_scan = sitk.ReadImage(file_path)
    ct_scan = sitk.GetArrayFromImage(ct_scan)
    return ct_scan, label

def _expand_dims(data, label):
        return tf.expand_dims(data, axis=3), label

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((metadata_test["seriesuid"].values, metadata_test["class"].values)) \
.map(
    lambda x, y: tf.py_function(
        process,
        [x, y],
        Tout=(tf.float32, tf.int64))) \
.map(lambda x, y: _expand_dims(x, y))

In [7]:
for data, label in dataset.batch(5).take(1):
    data = data